In [1]:
from github import Github, GithubException
import pandas as pd
import yaml
import yaml
import requests
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException


In [4]:
def cofepris(medicamento):
    url="https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options=webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')


    driver=webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    combo=driver.find_element(By.CLASS_NAME, "form-control")
    como_select=Select(combo)
    como_select.select_by_visible_text("Denominación Distintiva")
    #Escribir búsqueda en cuadro de texto
    cuadro_busqueda=driver.find_element(By.ID, "MainContent_txtSearchValue")
    #Medicamento a buscar
    cuadro_busqueda.send_keys(medicamento)
    #Boton de búsqueda
    button = driver.find_element(By.XPATH,"//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
    #Click para buscar
    button.click()
    try:
        # Encontrar elementos de la tabla.
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
    except NoSuchElementException:
        # No se encontraron resultados. Se regresa un dataframe vacío.
        driver.quit()
        return pd.DataFrame({'marca': [], 'generica': [], 'laboratorio': [], 'fabricante': []})
    #driver.quit()
    # Sopa
    soup = BeautifulSoup(table_contents, 'html.parser')
  
  #Extraer los datos necesarios
    marca=[]
    generica=[]
    lab=[]
    fabricante=[]


    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)

                #Dataframe
    df=pd.DataFrame({'marca':marca, 'generica':generica, 'laboratorio':lab, 'fabricante':fabricante})
    #Todas las variables a minúsculas
    df=df.apply(lambda x: x.astype(str).str.lower())
    #Eliminar \n
    df=df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    return df

## Ejemplo

In [5]:
lista_meds=["actron","histiacil"]

In [6]:
df=pd.DataFrame()
for med in lista_meds:
    df=pd.concat([df,cofepris(med)])

In [7]:
df

,marca,generica,laboratorio,fabricante
0,actron,ibuprofeno,bayer de mexico s.a. de c.v.,
1,actron plus,cafeína / ibuprofeno,"bayer de mexico, s.a. de c.v.",catalent argentina s.a.i.c.
2,actron,ibuprofeno,"bayer de mexico, s.a. de c.v.","berlimed, s.a."
0,histiacil nf,dextrometorfano / ambroxol,"sanofi-aventis de méxico, s. a. de c. v.","gelcaps exportadora de méxico, s. a. de c. v"
1,histiacil natix,hedera hélix,sanofi-aventis de méxico s.a. de c.v.,"sanofi-aventis de méxico, s.a. de c.v."
2,histiacil flu,cafeína / fenilefrina / paracetamol,sanofi-aventis de méxico s.a. de c.v.,"sanofi-aventis de méxico, s.a. de c.v."
3,histiacil fam,guaifenesina/oxolamina,"sanofi aventis de mexico, s.a. de c.v.","sanofi-aventis de méxico, s.a. de c.v."
4,histiacil-fam,guaifenesina / oxolamina,"sanofi aventis de mexico, s.a. de c.v.","sanofi-aventis de méxico, s.a. de c.v."
